In [1]:
import os

# 환경 변수 설정
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [2]:
import pandas as pd
import numpy as np
import random

import gymnasium as gym
from gymnasium import spaces
import pybamm

from stable_baselines3 import DDPG, PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

In [3]:
params = pybamm.ParameterValues("Chen2020").copy()

In [4]:
params

{'Ambient temperature [K]': 298.15,
 'Boltzmann constant [J.K-1]': 1.380649e-23,
 'Bulk solvent concentration [mol.m-3]': 2636.0,
 'Cation transference number': 0.2594,
 'Cell cooling surface area [m2]': 0.00531,
 'Cell thermal expansion coefficient [m.K-1]': 1.1e-06,
 'Cell volume [m3]': 2.42e-05,
 'Contact resistance [Ohm]': 0,
 'Current function [A]': 5.0,
 'EC diffusivity [m2.s-1]': 2e-18,
 'EC initial concentration in electrolyte [mol.m-3]': 4541.0,
 'Electrode height [m]': 0.065,
 'Electrode width [m]': 1.58,
 'Electrolyte conductivity [S.m-1]': <function electrolyte_conductivity_Nyman2008 at 0x0000026D86F71F30>,
 'Electrolyte diffusivity [m2.s-1]': <function electrolyte_diffusivity_Nyman2008 at 0x0000026D86F71EA0>,
 'Electron charge [C]': 1.602176634e-19,
 'Faraday constant [C.mol-1]': 96485.33212,
 'Ideal gas constant [J.K-1.mol-1]': 8.314462618,
 'Initial concentration in electrolyte [mol.m-3]': 1000.0,
 'Initial concentration in negative electrode [mol.m-3]': 29866.0,
 'Initi

In [5]:
class DFN(gym.Env):
    def __init__(self, render_mode=None):
        options = {"thermal": "lumped"}
        self.model = pybamm.lithium_ion.SPMe(options)

        self.params = pybamm.ParameterValues("Chen2020").copy()
        init_input = {
            'Number of cells connected in series to make a battery': 96,
            'Upper voltage cut-off [V]': 5,
        }
        self.params.update(init_input)
        
        geometry = self.model.default_geometry 
        submesh_types = self.model.default_submesh_types 
        var_pts = self.model.default_var_pts 
        spatial_methods = self.model.default_spatial_methods 
        
        self.params.process_geometry(geometry)
        mesh = pybamm.Mesh(geometry, submesh_types, var_pts) 
        self.disc = pybamm.Discretisation(mesh, spatial_methods)

        self.solutions = []

        self.r_max_temp = 273 + 35
        self.r_max_volt = 4.2
        self.SoC_desired = 0.8

        self.volt = 0
        self.temp = 0
        self.SoC = 0
        
        self.time_goal = 0
        self.ep_num = 0
        self.time_step = 0
        self.MAX_time_step = 3600*2

        self.observation_space = spaces.Box(low=0, high=400, shape=(4,), dtype=np.float32)
        self.action_space = spaces.Box(dtype=np.float32, low=-1000, high=-0, shape=(1,))
        
    def make_new_model(self, update_input):
        model1 = self.model.new_copy()
        param1 = self.params.copy()
        
        param1.update(update_input)
        model1 = param1.process_model(model1, inplace=False) # 파라미터에 모델 변화
        built_model = self.disc.process_model(model1, inplace=True, check_model=True) # 모델을 프로세싱
        return built_model
    
    def update_model_step(self,input):
        model = self.make_new_model(input)
        solver = pybamm.CasadiSolver(mode="fast")
        if len(self.solutions) == 0:
            self.SoC = 0.2
            experiment = pybamm.Experiment(["Rest for 30 min"])
            sim = pybamm.Simulation(self.model, experiment=experiment)
            step_solution = sim.solve(initial_soc=0.2)
        else:
            
            step_solution = solver.step(self.solutions[-1].last_state,
                                        model,
                                        30,
                                        npts=30,
                                        save=False,)
        self.solutions += [step_solution]
        return step_solution

    def step(self, action):
            
        new_input = {
            "Current function [A]": float(action)
        }
        try:
            solution = self.update_model_step(new_input)
            
            self.temp = solution["X-averaged cell temperature [K]"].entries[-1]
            self.volt = solution["Terminal voltage [V]"].entries[-1]     
            Q = self.params["Nominal cell capacity [A.h]"]
            DC = solution["Discharge capacity [A.h]"].entries[-1]
            self.SoC = self.SoC-DC/Q

            if self.SoC >= self.SoC_desired:
                terminated = True

            else:
                terminated = False
            # Calculate reward based on various factors
            if self.time_step >= self.MAX_time_step:
                terminated = True
                reward = -10000
            
            r_temp = -5 * abs(self.temp - (273+35)) if self.temp> (273+35) else 0
            r_volt = -200 * abs(self.volt - self.r_max_volt) if self.volt > self.r_max_volt else 0

            r_step = -0.1 #if self.time_step >= self.time_goal else -0.1*abs(self.time_goal - self.time_step)
            #r_soc =  -10 * abs(self.SoC - self.SoC_desired + 1) if self.time_step  > self.time_goal  else 0 
            if self.time_step <= self.time_goal:
                r_soc = 10*(self.SoC - ((0.6/self.time_goal)*self.time_step+0.2))
            else :
                r_soc = (self.time_goal - self.time_step)
            print(r_temp,r_volt,r_step,r_soc)
            
            reward = r_step +r_temp +r_volt + r_soc 

            reward = float(reward)               
            # Check if termination condition is met
            
            self.time_step +=1
        except:
            terminated = True
            reward = -10000

        observation = self._get_obs()
        info = self._get_info()
        print(self.time_step , observation,reward,"|",float(action),"|")
        return observation, reward, terminated, False, info

    def reset(self, seed=None, options=None):
        print(self.time_step)
        self.time_goal = self.generate_random_number()
        super().reset(seed=seed)
        print("reset==================================")
        print(self.time_goal)
        self.solutions = []
        self.SoC = 0.2
        experiment = pybamm.Experiment(["Rest for 30 min"])
        sim = pybamm.Simulation(self.model, experiment=experiment)
        step_solution = sim.solve(initial_soc=0.2)
        self.solutions += [step_solution]
        observation = self._get_obs()
        info = self._get_info()
        self.ep_num +=1
        self.time_step = 0
        return observation, info
    
    def generate_random_number(self):
        return int(random.random()*100+20)*2 

    def _get_obs(self):
        return np.array([self.SoC,self.volt,self.temp,self.time_goal-self.time_step], dtype=np.float32)

    def _get_info(self):
        return {"distance": self.SoC_desired - self.SoC}

env = DFN()
# It will check your custom environment and output additional warnings if needed
check_env(env)  

0
reset==================================
202
0
reset==================================
122
0 [  0.2   0.    0.  122. ] -10000 | -975.4620971679688 |
0
reset==================================
190
0 [  0.2   0.    0.  190. ] -10000 | -575.2017211914062 |
0
reset==================================
120
0 [  0.2   0.    0.  120. ] -10000 | -299.74407958984375 |
0
reset==================================
148
0 [  0.2   0.    0.  148. ] -10000 | -941.0272827148438 |
0
reset==================================
176
0 [  0.2   0.    0.  176. ] -10000 | -966.8298950195312 |
0
reset==================================
46
0 [ 0.2  0.   0.  46. ] -10000 | -265.8564453125 |
0
reset==================================
128
0 [  0.2   0.    0.  128. ] -10000 | -411.6342468261719 |
0
reset==================================
46
0 [ 0.2  0.   0.  46. ] -10000 | -511.5181579589844 |
0
reset==================================
168
0 0 -0.1 0.16112645466585307
1 [2.1611264e-01 3.8324046e+00 2.9954987e+02 1.6700000e+02]

In [6]:
# The noise objects for DDPG


model = PPO("MlpPolicy", env,  verbose=1)
model.learn(total_timesteps=300000, log_interval=10)
model.save("ddpg_pendulum")
vec_env = model.get_env()


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
3
reset==================================
148
0 0 -0.1 0.0
1 [2.0000000e-01 3.5285187e+00 2.9814999e+02 1.4700000e+02] -0.1 | 0.0 |
0 0 -0.1 -0.031156263077894153
2 [2.0093843e-01 3.5540359e+00 2.9815659e+02 1.4600000e+02] -0.13115626307789416 | -0.5630566477775574 |
0 0 -0.1 -0.07169680361843456
3 [2.0093843e-01 3.5338264e+00 2.9815637e+02 1.4500000e+02] -0.17169680361843456 | 0.0 |
0 0 -0.1 -0.11223734415897496
4 [2.0093843e-01 3.5332880e+00 2.9815613e+02 1.4400000e+02] -0.21223734415897497 | 0.0 |
0 0 -0.1 -0.15277788469951564
5 [2.0093843e-01 3.5330369e+00 2.9815591e+02 1.4300000e+02] -0.2527778846995157 | 0.0 |
0 0 -0.1 -0.1796131716394389
6 [2.0230895e-01 3.5676947e+00 2.9817154e+02 1.4200000e+02] -0.2796131716394389 | -0.8223152160644531 |
0 0 -0.1 -0.20932154153494448
7 [2.0339216e-01 3.5646873e+00 2.9818167e+02 1.4100000e+02] -0.3093215415349445 | -0.6499302387237549 |
0 0 -0.1 -0.249